## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description
0,0,Hermanus,ZA,-34.42,19.23,61.00,81,86,12.95,overcast clouds
1,1,Akdepe,TM,42.06,59.38,53.60,41,0,9.17,clear sky
2,2,Saint-Philippe,RE,-21.36,55.77,69.80,73,0,5.82,clear sky
3,3,Galle,LK,6.04,80.22,78.85,85,100,7.52,overcast clouds
4,4,Bredasdorp,ZA,-34.53,20.04,62.60,82,100,5.82,overcast clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description
3,3,Galle,LK,6.04,80.22,78.85,85,100,7.52,overcast clouds
20,20,Butaritari,KI,3.07,172.79,81.18,70,72,9.51,broken clouds
23,23,Gondanglegi,ID,-7.75,112.01,76.01,86,99,9.17,overcast clouds
24,24,Isangel,VU,-19.55,169.27,77.31,70,7,10.63,clear sky
33,33,Coahuayana,MX,18.73,-103.68,86.00,66,40,9.17,scattered clouds
53,53,Mahibadhoo,MV,3.78,72.97,82.06,79,100,8.28,overcast clouds
55,55,Sabang,ID,5.89,95.32,82.24,74,100,16.06,overcast clouds
58,58,Atuona,PF,-9.80,-139.03,78.93,75,0,19.75,clear sky
60,60,Lamu,KE,-2.27,40.90,80.60,78,20,4.70,few clouds
61,61,Hithadhoo,MV,-0.60,73.08,82.99,70,37,5.66,scattered clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().count()

City_ID        157
City           157
Country        157
Lat            157
Lng            157
Max Temp       157
Humidity       157
Cloudiness     157
Wind Speed     157
Description    157
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df=preferred_cities_df.dropna()

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
3,Galle,LK,78.85,overcast clouds,6.04,80.22,
20,Butaritari,KI,81.18,broken clouds,3.07,172.79,
23,Gondanglegi,ID,76.01,overcast clouds,-7.75,112.01,
24,Isangel,VU,77.31,clear sky,-19.55,169.27,
33,Coahuayana,MX,86.00,scattered clouds,18.73,-103.68,
53,Mahibadhoo,MV,82.06,overcast clouds,3.78,72.97,
55,Sabang,ID,82.24,overcast clouds,5.89,95.32,
58,Atuona,PF,78.93,clear sky,-9.80,-139.03,
60,Lamu,KE,80.60,few clouds,-2.27,40.90,
61,Hithadhoo,MV,82.99,scattered clouds,-0.60,73.08,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [9]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df=hotel_df.dropna()

In [10]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_Vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [11]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Country</dt><dd>{Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [12]:
# 11a. Add a marker layer for each city to the map. 
locations = clean_hotel_df[["Lat", "Lng"]]
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))